In [ ]:
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Load datasets
try:
    ratings_df = pd.read_csv("ratings.csv")
    movies_df = pd.read_csv("movies.csv")
    print("Successfully loaded MovieLens dataset")
except FileNotFoundError:
    print("ERROR: Dataset files not found.")
    exit()

In [2]:
ratings = {}
for i in range(len(ratings_df)):
    u = ratings_df["userId"][i] - 1
    m = ratings_df["movieId"][i] - 1
    r = ratings_df["rating"][i]
    ratings[(u, m)] = r

data = [(user, movie, rating) for (user, movie), rating in ratings.items()]
train_ratings, test_ratings = train_test_split(data, test_size=0.2, random_state=10)

In [3]:
# ratings input: list of (i, j, r_ij) tuples
def model(ratings, n_users, m_movies, k):
    # Priors for user latent factors and bias
    with pyro.plate("users", n_users):
        nu = pyro.sample("nu", dist.Normal(0., 0.1))
        U = pyro.sample("U", dist.MultivariateNormal(torch.zeros(k), torch.eye(k)))
    
    # Priors for movie latent factors and bias
    with pyro.plate("movies", m_movies):
        mu = pyro.sample("mu", dist.Normal(0., 0.316))
        V = pyro.sample("V", dist.MultivariateNormal(torch.zeros(k), torch.eye(k)))
    
    sigma = 0.5  # observation noise

    # Plate over observed ratings
    with pyro.plate("observe_data", len(ratings)):
        user = torch.tensor([r[0] for r in ratings])
        movie = torch.tensor([r[1] for r in ratings])
        observed_rating = torch.tensor([r[2] for r in ratings])

        pred = nu[user] + mu[movie] + torch.sum(U[user] * V[movie], dim=1)
        pyro.sample("obs", dist.Normal(pred, sigma), obs=observed_rating)

In [4]:
def guide(ratings, n_users, m_movies, k):
    # User bias and latent vectors
    nu_loc = pyro.param("nu_loc", torch.zeros(n_users))
    nu_scale = pyro.param("nu_scale", torch.ones(n_users), constraint=dist.constraints.positive)

    U_loc = pyro.param("U_loc", torch.randn(n_users, k))
    U_cov = pyro.param("U_cov", torch.stack([0.1 * torch.eye(k) for _ in range(n_users)]),
                       constraint=dist.constraints.positive_definite)
    with pyro.plate("users", n_users):
        pyro.sample("nu", dist.Normal(nu_loc, nu_scale))
        pyro.sample("U", dist.MultivariateNormal(U_loc, U_cov.to(dtype=torch.float32)))

    # Movie bias and latent vectors
    mu_loc = pyro.param("mu_loc", torch.zeros(m_movies))
    mu_scale = pyro.param("mu_scale", torch.ones(m_movies), constraint=dist.constraints.positive)

    V_loc = pyro.param("V_loc", torch.randn(m_movies, k))
    V_cov = pyro.param("V_cov", torch.stack([0.1 * torch.eye(k) for _ in range(m_movies)]),
                       constraint=dist.constraints.positive_definite)
    with pyro.plate("movies", m_movies):
        pyro.sample("mu", dist.Normal(mu_loc, mu_scale))
        pyro.sample("V", dist.MultivariateNormal(V_loc, V_cov.to(dtype=torch.float32)))

In [5]:
def train_svi(train_ratings, n_users, m_movies, k, num_steps=1500, lr=0.01):
    pyro.clear_param_store()
    optimizer = Adam({"lr": lr})
    svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

    for step in range(num_steps):
        loss = svi.step(train_ratings, n_users, m_movies, k)
        if (step + 1) % 100 == 0:
            print(f"Step {step} - ELBO loss: {loss:.2f}")

def predict(i, j, k):
    # Extract learned variational parameters
    nu_i = pyro.param("nu_loc")[i]
    mu_j = pyro.param("mu_loc")[j]
    U_i = pyro.param("U_loc")[i]
    V_j = pyro.param("V_loc")[j]
    return (U_i @ V_j + nu_i + mu_j).item()

def predict_with_uncertainty(i, j, k):
    # Means
    nu_i = pyro.param("nu_loc")[i]
    mu_j = pyro.param("mu_loc")[j]
    U_i = pyro.param("U_loc")[i]
    V_j = pyro.param("V_loc")[j]

    # Covariances
    U_cov_i = pyro.param("U_cov")[i]
    V_cov_j = pyro.param("V_cov")[j]

    # Predictive mean
    pred_mean = (U_i @ V_j + nu_i + mu_j)

    var = (
        U_i @ V_cov_j @ U_i
        + V_j @ U_cov_i @ V_j
        + 0.25  # observation noise squared (sigma^2 = 0.5^2)
    )
    return pred_mean.item(), torch.sqrt(var).item()

def compute_rmse(test_ratings, k):
    preds = []
    trues = []
    for i, j, r in test_ratings:
        preds.append(predict(i, j, k))
        trues.append(r)
    preds = torch.tensor(preds)
    trues = torch.tensor(trues)
    return torch.sqrt(torch.mean((preds - trues) ** 2)).item()

# Train model
n_users = ratings_df["userId"].max()
m_movies = ratings_df['movieId'].max()
k = 2 # Latent dimensions

train_svi(train_ratings, n_users, m_movies, k)

# Evaluate
rmse = compute_rmse(test_ratings, k)
print(f"Test RMSE: {rmse:.4f}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71


KeyboardInterrupt: 